###  Created by Luis Alejandro (alejand@umich.edu)

Extracts TMID info from all signals into DB. **Careful!!! This will overwrite all entries in DB!!!**

**Install the following packages:**

pip install npTDMS<br>
pip install mysql-connector-python<br>

In [1]:
import mysql.connector
from nptdms import TdmsFile
import numpy as np
from scipy.signal import spectrogram
from scipy.signal import decimate
from IPython.display import clear_output

In [2]:
# Connects to DB
mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "cuba",
        database = "airnoise"
    )
# Gets signals
dbcursor = mydb.cursor()
sql = '''SELECT t.measurement, t.array, t.microphone FROM
        tmid t ORDER BY t.measurement, t.array, t.microphone;'''
dbcursor.execute(sql)
signals = dbcursor.fetchall()

#### TMID Extraction
Tries to determine point tmid by computing the point in time where the maximum average spectrum power is achieved.

In [3]:
fs = 51200
factor = 6
curr_measurement = None
for k, (measurement,array,microphone) in enumerate(signals):
    if measurement != curr_measurement:
        dbcursor = mydb.cursor()        
        sql = '''SELECT m.url FROM measurements m WHERE
                    m.id_measurement = {:}'''.format(measurement)
        dbcursor.execute(sql)
        results = dbcursor.fetchall()
        url = results[0][0]
        tdms = TdmsFile("../../datasets/classification/noise/" + url)
        curr_measurement = measurement
    data = tdms.channel_data(group = 'Untitled', channel = 'cDAQ1Mod' + str(array) + '/ai' + str(microphone - 1))
    data = data/max(np.absolute(data))
    data = decimate(data,factor)
    f, t, Sxx = spectrogram(data,int(fs/factor),mode='magnitude',
                                window='hamming',nperseg=512,noverlap=256)
    power = Sxx.mean(axis=0)
    location = t[power.argmax()]
    dbcursor = mydb.cursor()
    sql = '''UPDATE tmid t SET t.location = {} WHERE t.measurement = {}
            AND t.array = {} AND t.microphone = {}'''.format(
            location,measurement,array,microphone)
    dbcursor.execute(sql)
    mydb.commit()
    print('Updating measurement: {}, array: {}, microphone: {} with location {}'.format(measurement,array,microphone,location))
    clear_output(True)

Updating measurement: 189, array: 3, microphone: 4 with location 12.990507441696941


#### TMID Correction
For some signals within a measurement the previous proceduce fails, so we try to correct those cases by looking to the results of the other signals within that same measurement. If we detect a mistake we correct tmid using the median of all signals within the measurement.

In [5]:
# Gets measurements ids
dbcursor = mydb.cursor()
sql = '''SELECT m.id_measurement FROM measurements m ORDER BY m.id_measurement;'''
dbcursor.execute(sql)
measurements = np.array([entry[0] for entry in dbcursor.fetchall()])
for measurement in measurements:
    dbcursor = mydb.cursor()        
    sql = '''SELECT t.array, t.microphone, t.location FROM tmid t WHERE t.measurement = {:}'''.format(measurement)
    dbcursor.execute(sql)
    entries = dbcursor.fetchall()
    locations = np.array([loc for _,_,loc in entries])
    mean = locations.mean()
    threshold = locations.std()
    for array,microphone,location in entries:
        if location < mean - threshold or location > mean + threshold:
            dbcursor = mydb.cursor()
            update = np.median(locations)
            sql = '''UPDATE tmid t SET t.location = {} WHERE t.measurement = {}
                AND t.array = {} AND t.microphone = {}'''.format(
                update,measurement,array,microphone)
            dbcursor.execute(sql)
            mydb.commit()
            print('Updating measurement: {}, array: {}, microphone: {} from location {} to location {}'.format(
                measurement,array,microphone,location,update))

Updating measurement: 1, array: 3, microphone: 1 from location 10.5604 to location 10.9204
Updating measurement: 1, array: 3, microphone: 2 from location 10.5604 to location 10.9204
Updating measurement: 1, array: 3, microphone: 3 from location 10.5604 to location 10.9204
Updating measurement: 1, array: 2, microphone: 4 from location 10.5004 to location 10.9204
Updating measurement: 1, array: 3, microphone: 4 from location 10.5604 to location 10.9204
Updating measurement: 2, array: 2, microphone: 2 from location 11.3404 to location 11.7605
Updating measurement: 2, array: 2, microphone: 4 from location 11.3404 to location 11.7605
Updating measurement: 3, array: 1, microphone: 1 from location 11.2204 to location 10.8904
Updating measurement: 3, array: 2, microphone: 1 from location 10.2604 to location 10.8904
Updating measurement: 3, array: 1, microphone: 2 from location 11.2204 to location 10.8904
Updating measurement: 3, array: 1, microphone: 3 from location 11.2204 to location 10.8904

Updating measurement: 54, array: 1, microphone: 2 from location 13.8605 to location 14.4906
Updating measurement: 54, array: 1, microphone: 3 from location 13.8605 to location 14.4906
Updating measurement: 54, array: 1, microphone: 4 from location 13.7105 to location 14.4906
Updating measurement: 55, array: 3, microphone: 4 from location 1.59006 to location 13.4105
Updating measurement: 56, array: 2, microphone: 1 from location 5.28021 to location 11.6405
Updating measurement: 56, array: 1, microphone: 3 from location 0.960038 to location 11.6405
Updating measurement: 56, array: 3, microphone: 4 from location 16.5906 to location 11.6405
Updating measurement: 57, array: 1, microphone: 1 from location 8.88035 to location 9.36037
Updating measurement: 57, array: 2, microphone: 1 from location 8.73034 to location 9.36037
Updating measurement: 57, array: 3, microphone: 4 from location 8.88035 to location 9.36037
Updating measurement: 58, array: 1, microphone: 1 from location 7.6503 to locat

Updating measurement: 98, array: 1, microphone: 2 from location 10.2904 to location 10.6504
Updating measurement: 98, array: 3, microphone: 2 from location 10.8304 to location 10.6504
Updating measurement: 98, array: 1, microphone: 3 from location 10.2904 to location 10.6504
Updating measurement: 98, array: 3, microphone: 3 from location 10.8304 to location 10.6504
Updating measurement: 99, array: 1, microphone: 1 from location 11.6405 to location 11.7005
Updating measurement: 99, array: 3, microphone: 1 from location 11.9405 to location 11.7005
Updating measurement: 99, array: 1, microphone: 2 from location 11.6405 to location 11.7005
Updating measurement: 99, array: 3, microphone: 2 from location 11.9405 to location 11.7005
Updating measurement: 99, array: 3, microphone: 3 from location 11.9705 to location 11.7005
Updating measurement: 99, array: 1, microphone: 4 from location 11.9105 to location 11.7005
Updating measurement: 100, array: 1, microphone: 2 from location 2.6701 to locat

Updating measurement: 137, array: 1, microphone: 1 from location 15.7506 to location 15.7356
Updating measurement: 137, array: 3, microphone: 1 from location 15.7506 to location 15.7356
Updating measurement: 137, array: 1, microphone: 2 from location 15.7506 to location 15.7356
Updating measurement: 137, array: 3, microphone: 2 from location 15.7506 to location 15.7356
Updating measurement: 137, array: 1, microphone: 3 from location 15.7506 to location 15.7356
Updating measurement: 137, array: 3, microphone: 3 from location 15.7506 to location 15.7356
Updating measurement: 138, array: 1, microphone: 1 from location 13.5005 to location 13.1105
Updating measurement: 138, array: 1, microphone: 2 from location 13.5005 to location 13.1105
Updating measurement: 138, array: 3, microphone: 3 from location 13.5005 to location 13.1105
Updating measurement: 139, array: 2, microphone: 1 from location 12.1505 to location 11.3404
Updating measurement: 139, array: 2, microphone: 2 from location 12.15

Updating measurement: 177, array: 2, microphone: 1 from location 12.5105 to location 12.5405
Updating measurement: 177, array: 3, microphone: 1 from location 12.5105 to location 12.5405
Updating measurement: 177, array: 1, microphone: 2 from location 12.5105 to location 12.5405
Updating measurement: 177, array: 1, microphone: 3 from location 12.5105 to location 12.5405
Updating measurement: 177, array: 3, microphone: 3 from location 12.5105 to location 12.5405
Updating measurement: 178, array: 2, microphone: 2 from location 11.1004 to location 8.97035
Updating measurement: 179, array: 3, microphone: 3 from location 8.25032 to location 10.4404
Updating measurement: 180, array: 2, microphone: 2 from location 15.9306 to location 12.3605
Updating measurement: 181, array: 2, microphone: 1 from location 12.3005 to location 11.1004
Updating measurement: 181, array: 2, microphone: 4 from location 12.3305 to location 11.1004
Updating measurement: 182, array: 2, microphone: 2 from location 12.39